# HDS (Historical Data Set) Analysis

This notebook demonstrates how to work with a sample HDS dataset, explore its structure, and build a simple XGBoost model to show feature importance.

It then shows how to combine this data with a sample dataset from the Data Lake and perform an analysis of which features could be considered to pull in to Pega to improve the models.

In [1]:
import polars as pl
import polars.selectors as cs
import plotly.express as px
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, roc_auc_score
from pdstools.utils import cdh_utils

## Data Read

Read the HDS data. Depending on how the data was extracted and stored, you may need different reading methods using
zipfile and/or Polars.

We're also casting the data to the appropriate types and dropping some features that shouldnt be used for models. 

In [2]:
archive = zipfile.ZipFile("../../data/hds.zip", "r")
hds_data = (
    pl.concat([pl.read_ndjson(archive.open(f)) for f in archive.namelist()])
    .rename({"Customer_C_CIFNBR": "Customer_ID"})
    .with_columns(
        cdh_utils.parse_pega_date_time_formats("Decision_DecisionTime"),
        cdh_utils.parse_pega_date_time_formats("Decision_OutcomeTime"),
        cs.ends_with("_DaysSince", "_pyHistoricalOutcomeCount").cast(pl.Float64),
        pl.col(
            [
                "Customer_NetWealth",
                "Customer_CreditScore",
                "Customer_CLV_VALUE",
                "Customer_RelationshipStartDate",
                "Customer_Date_of_Birth",
                "Customer_NoOfDependents"
            ]
        ).cast(pl.Float64),
        cs.starts_with("Param_ExtGroup").cast(pl.Float64),
    )
    .drop(["Customer_Gender", "Customer_Prefix"])
)
hds_data.describe()

statistic,Customer_CLV,Customer_pyCountry,Customer_RiskCode,IH_Web_Inbound_Rejected_pxLastOutcomeTime_DaysSince,Decision_SubjectID,Decision_Outcome,Customer_City,IH_Web_Inbound_Loyal_pxLastOutcomeTime_DaysSince,Context_Name,Decision_OutcomeTime,Customer_RelationshipStartDate,Customer_ID,IH_Web_Inbound_Rejected_pyHistoricalOutcomeCount,Param_ExtGroupCreditcards,Customer_Date_of_Birth,Customer_BusinessSegment,Customer_NoOfDependents,Customer_CreditScore,Decision_InteractionID,Customer_Incarceration,IH_Email_Outbound_Rejected_pxLastGroupID,Customer_LastReviewedDate,IH_Web_Inbound_Accepted_pxLastGroupID,IH_Email_Outbound_Accepted_pxLastOutcomeTime_DaysSince,IH_SMS_Outbound_Loyal_pxLastGroupID,rulesetVersion,IH_Email_Outbound_Churned_pyHistoricalOutcomeCount,Context_Issue,Customer_ReviewDate,Customer_WinScore,Context_Group,IH_Web_Inbound_Churned_pyHistoricalOutcomeCount,Customer_InCollections,IH_SMS_Outbound_Accepted_pyHistoricalOutcomeCount,IH_Email_Outbound_Loyal_pyHistoricalOutcomeCount,Customer_Age,…,IH_SMS_Outbound_Accepted_pxLastOutcomeTime_DaysSince,Decision_DecisionTime,Customer_MilitaryService,IH_SMS_Outbound_Churned_pxLastGroupID,Customer_Bankruptcy,Customer_RiskScore,IH_Web_Inbound_Rejected_pxLastGroupID,IH_Web_Inbound_Loyal_pyHistoricalOutcomeCount,IH_Email_Outbound_Churned_pxLastGroupID,Customer_State,IH_Email_Outbound_Rejected_pyHistoricalOutcomeCount,Context_Direction,Customer_NaturalDisaster,Customer_CLV_VALUE,Customer_MaritalStatus,Customer_AnnualIncome,Customer_NextReviewDate,IH_Email_Outbound_Churned_pxLastOutcomeTime_DaysSince,IH_SMS_Outbound_Loyal_pyHistoricalOutcomeCount,IH_SMS_Outbound_Loyal_pxLastOutcomeTime_DaysSince,IH_SMS_Outbound_Churned_pxLastOutcomeTime_DaysSince,Customer_TotalLiabilities,Context_Channel,id,Customer_ResidentialStatus,Decision_Rank,IH_SMS_Outbound_Rejected_pyHistoricalOutcomeCount,rulesetName,IH_Web_Inbound_Accepted_pyHistoricalOutcomeCount,IH_Email_Outbound_Loyal_pxLastGroupID,Param_ExtGroupWealthoffers,Customer_OrganizationLabel,Param_ExtGroupAutoloans,IH_Web_Inbound_Churned_pxLastOutcomeTime_DaysSince,Customer_pyRegion,IH_SMS_Outbound_Accepted_pxLastGroupID,Customer_Deceased
str,str,str,str,f64,str,str,str,f64,str,str,f64,str,f64,f64,f64,str,f64,f64,str,str,str,str,str,f64,str,str,f64,str,str,str,str,f64,str,f64,f64,str,…,f64,str,str,str,str,str,str,f64,str,str,f64,str,str,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,str,f64,str,f64,str,f64,str,f64,f64,str,str,str
"""count""","""12062""","""12062""","""12062""",12062.0,"""12062""","""12062""","""12062""",6076.0,"""12062""","""12062""",12062.0,"""12062""",12062.0,12062.0,12062.0,"""12062""",12062.0,12062.0,"""12062""","""12062""","""12062""","""12062""","""11971""",8156.0,"""1169""","""12062""",2259.0,"""12062""","""12062""","""12062""","""12062""",5911.0,"""12062""",11826.0,2432.0,"""11947""",…,11826.0,"""12062""","""12062""","""828""","""12062""","""12062""","""12062""",6076.0,"""2259""","""12062""",12062.0,"""12062""","""12062""",12062.0,"""12062""","""12062""","""12062""",2259.0,1169.0,1169.0,828.0,"""12062""","""12062""","""12062""","""12062""","""12062""",12062.0,"""12062""",11971.0,"""2432""",12062.0,"""12062""",12062.0,5911.0,"""12062""","""11826""","""12062"""
"""null_count""","""0""","""0""","""0""",0.0,"""0""","""0""","""0""",5986.0,"""0""","""0""",0.0,"""0""",0.0,0.0,0.0,"""0""",0.0,0.0,"""0""","""0""","""0""","""0""","""91""",3906.0,"""10893""","""0""",9803.0,"""0""","""0""","""0""","""0""",6151.0,"""0""",236.0,9630.0,"""115""",…,236.0,"""0""","""0""","""11234""","""0""","""0""","""0""",5986.0,"""9803""","""0""",0.0,"""0""","""0""",0.0,"""0""","""0""","""0""",9803.0,10893.0,10893.0,11234.0,"""0""","""0""","""0""","""0""","""0""",0.0,"""0""",91.0,"""9630""",0.0,"""0""",0.0,6151.0,"""0""","""236""","""0"""
"""mean""",null,null,null,17.320495,null,null,null,29.931684,null,"""2021-05-24 02:01:33.432064""",1954.04534,null,50.346211,0.207214,18775.43305,null,2.091527,618.270602,null,null,null,null,null,51.498683,null,null,1.

## Available Fields in the HDS dataset

The HDS data contains all the payload sent to the ADM models (over a period of time) plus the outcomes (Accepted/Declined/Clicked etc). There are a few categories of fields, that can be identified by their prefix:

* "Customer" fields, representing the fields/predictors configured in ADM
* "Context" fields, these are Channel/Direction/Issue/Group/Name, the usual "context identifiers" of the ADM models
* "IH" fields, these are Pega-generated fields derived from Interaction History
* Optional "Param" fields, also user defined fields/predictors, but configured in the strategies, rather than defined in the ADM model configuration

Meta information about the decisions is in Decision and internal fields, containing info about the time of decision, the sample size etc. These are not used in the models.



In [3]:
hds_data_dictionary = (
    pl.DataFrame(
           {"Field" : hds_data.schema.names(),
           "Numeric" : [x.is_numeric() for x in hds_data.schema.dtypes()],}
    )
    .with_columns(
        Category=pl.when(pl.col("Field").str.contains("_", literal=True))
        .then(pl.col("Field").str.replace(r"([^_]+)_.*", "${1}"))
        .otherwise(pl.lit("Internal"))
    )
    .sort("Category")
)
hds_data_dictionary.to_pandas().style.hide()


Field,Numeric,Category
Context_Name,False,Context
Context_Issue,False,Context
Context_Group,False,Context
Context_Direction,False,Context
Context_Channel,False,Context
Customer_CLV,False,Customer
Customer_pyCountry,False,Customer
Customer_RiskCode,False,Customer
Customer_City,False,Customer
Customer_RelationshipStartDate,True,Customer


In [4]:
category_counts = (
    hds_data_dictionary
    .group_by("Category", "Numeric")
    .agg(Count=pl.len())
    .sort("Category")
)
fig = px.bar(
    category_counts, #.to_dict(as_series=False),
    y="Category",
    x="Count",
    title="Number of Fields by Category",
    color="Numeric",
    text="Count",
    orientation="h",
)

fig.update_layout(
    yaxis_title="Field Category",
    xaxis_title="Number of Fields",
)

fig.show()

## Create XGBoost model 

From this HDS data we create a simple XGBoost model. We create a simple model over all channels and all actions, not split up like ADM does. This could be changed, but the goal here is to get a list of the features ranked by importance, not to create a model that is better than ADM.

First, there is data prep to do one-hot encoding and target encoding.

In [5]:
def data_prep(data, data_dictionary):
    categorical_fields = (
        data_dictionary.filter(~pl.col("Numeric"))
        .filter((pl.col("Category") != "Internal") & (pl.col("Category") != "Decision"))
        .select("Field")
        .to_series()
        .to_list()
    )
    numerical_fields = (
        data_dictionary.filter(pl.col("Numeric"))
        .filter((pl.col("Category") != "Internal") & (pl.col("Category") != "Decision"))
        .select("Field")
        .to_series()
        .to_list()
    )

    print(f"Categorical fields: {categorical_fields}")
    print(f"Numerical fields: {numerical_fields}")

    # Simple encoding for categorical features
    for column in categorical_fields:
        if column != "Decision_Outcome":
            # Handle missing values
            data = data.with_columns(
                pl.col(column).fill_null("missing").alias(column)
            )

            # Create a simple label encoder
            le = LabelEncoder()
            encoded = le.fit_transform(data[column].to_list())
            data = data.with_columns(
                pl.Series(name=column + "_encoded", values=encoded)
            )

    # Encode target variable
    le_target = LabelEncoder()
    encoded_target = le_target.fit_transform(data["Decision_Outcome"].to_list())
    data = data.with_columns(
        pl.Series(name="target", values=encoded_target)
    )

    # Show target encoding
    target_mapping = dict(zip(le_target.classes_, range(len(le_target.classes_))))
    print(f"\nTarget encoding: {target_mapping}")

    # Select features and target
    feature_cols = [
        col for col in data.columns if col.endswith("_encoded")
    ] + numerical_fields

    X = data[feature_cols]
    y = data["target"]

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    print(f"\nTraining set size: {X_train.shape[0]} samples")
    print(f"Test set size: {X_test.shape[0]} samples")

    return X_train, X_test, y_train, y_test, le_target, feature_cols


X_train, X_test, y_train, y_test, target_encoder, feature_cols = data_prep(hds_data, hds_data_dictionary)

Categorical fields: ['Context_Name', 'Context_Issue', 'Context_Group', 'Context_Direction', 'Context_Channel', 'Customer_CLV', 'Customer_pyCountry', 'Customer_RiskCode', 'Customer_City', 'Customer_ID', 'Customer_BusinessSegment', 'Customer_Incarceration', 'Customer_LastReviewedDate', 'Customer_ReviewDate', 'Customer_WinScore', 'Customer_InCollections', 'Customer_Age', 'Customer_BalanceTransaction', 'Customer_TotalAssets', 'Customer_IsCustomerActive', 'Customer_HealthMatter', 'Customer_MilitaryService', 'Customer_Bankruptcy', 'Customer_RiskScore', 'Customer_State', 'Customer_NaturalDisaster', 'Customer_MaritalStatus', 'Customer_AnnualIncome', 'Customer_NextReviewDate', 'Customer_TotalLiabilities', 'Customer_ResidentialStatus', 'Customer_OrganizationLabel', 'Customer_pyRegion', 'Customer_Deceased', 'IH_Email_Outbound_Rejected_pxLastGroupID', 'IH_Web_Inbound_Accepted_pxLastGroupID', 'IH_SMS_Outbound_Loyal_pxLastGroupID', 'IH_Email_Outbound_Accepted_pxLastGroupID', 'IH_Web_Inbound_Churned_

In [6]:
def create_classifier(X_train, X_test, y_train, y_test, target_encoder):
    # Create and train the model
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)

    # Make predictions and evaluate
    y_pred = xgb_model.predict(X_test)
    print (f"Model AUC: {round(roc_auc_score(y_test,y_pred), 5)}")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))

    return xgb_model

classifier = create_classifier(X_train, X_test, y_train, y_test, target_encoder)

Model AUC: 0.54311

Classification Report:
              precision    recall  f1-score   support

    Accepted       0.24      0.12      0.16       180
    Rejected       0.93      0.97      0.95      2233

    accuracy                           0.91      2413
   macro avg       0.58      0.54      0.55      2413
weighted avg       0.88      0.91      0.89      2413



In [7]:
def show_feature_imp(classifier, data, feature_cols):
    importances = classifier.feature_importances_

    # Create a DataFrame for feature importances
    feature_importance_df = (
        pl.DataFrame({"Feature": feature_cols, "Importance": importances.tolist()})
        .with_columns(
            Feature=pl.when(pl.col("Feature").str.ends_with("_encoded"))
            .then(pl.col("Feature").str.replace(r"_encoded$", ""))
            .otherwise(pl.col("Feature"))
        )
        .with_columns(
            Category=pl.when(pl.col("Feature").str.contains("_", literal=True))
            .then(pl.col("Feature").str.replace(r"([^_]+)_.*", "${1}"))
            .otherwise(pl.lit("Internal"))
        )
        .sort("Importance", descending=True)
    )

    # Get top 20 features by importance
    top_features_df = feature_importance_df.head(20)

    # Get the ordered list of feature names
    feature_order = top_features_df["Feature"].to_list()

    # Correlation test of new features
    features_encoded_name = [
        f"{feat}_encoded" if f"{feat}_encoded" in feature_cols else feat
        for feat in feature_order
    ]   

    similar_features = []
    n = len(features_encoded_name)

    for i in range(n):
        for j in range(i + 1, n):
            col1 = features_encoded_name[i]
            col2 = features_encoded_name[j]

            correlation = data.select(pl.corr(col1, col2)).item()
            if abs(correlation) >= 0.95:
                found = False

                for i, tup in enumerate(similar_features):
                    if col1 in tup:
                        similar_features[i] = tup + (col2,)
                        found = True
                        break
                    elif col2 in tup:
                        similar_features[i] = tup + (col1,)
                        found = True

                if not found:
                    similar_features.append((col1, col2))
    

    # Creating the group label
    group_mapping = {}
    for i, group in enumerate(similar_features, 1):
        for feature in group:
            group_mapping[feature] = f"Group {i}"

    top_features_df = top_features_df.with_columns(
        pl.col("Feature").map_elements(lambda f: group_mapping.get(f, ""), return_dtype=pl.Utf8).alias("GroupLabel")
    )

    # Plot feature importances
    fig = px.bar(
        top_features_df,
        x="Importance",
        y="Feature",
        orientation="h",
        title="Feature Importance",
        color="Category",
        text="GroupLabel",
        color_discrete_map={
            "Context": "orange",
            "IH": "green",
            "Customer": "blue",
            "Param": "lightblue",
            "DataLake": "red",
            "Internal": "gray",
            "Decision": "purple",
        },
    )

    fig.update_layout(
        xaxis_title="Importance",
        yaxis_title="Feature",
        yaxis=dict(
            categoryorder="array",
            categoryarray=feature_order,
            autorange="reversed",
            dtick=1,
        ),
    )

    fig.update_traces(
        textposition="outside",
        textfont=dict(color="black", size=12),
    )

    fig.show()


show_feature_imp(classifier, X_train, feature_cols)

Features that are strongly correlated are shown with an indication of the group (e.g. "Group 1"). Colors are used to differentiate the different sources of the features. In this demo data set, you see that Group and Channel are very important features, as is expected.

# Finding new Features from the Data Lake

Now, suppose you have external data from your data lake that you want to consider adding to Pega to improve the performance of your models.

If you have such data, you can merge it with the HDS data and run the model again to see how these features fare against what ADM already uses.

Such data is typically time-stamped, so we need to be careful to only pull in data from before the decisions were made. 

## Create (fake) External Data

We first create an example of external data. All features are captured over time there, so there is a feature name, a timestamp, and a value.

This code (and resulting data) are just an example. You can use any data you want, we just highlight the structure.


In [8]:
import random

random.seed(101)
datalake_fake_data = hds_data.with_columns(
    DataLake_BadFeature=pl.Series([random.random() for _ in range(hds_data.height)]),
    DataLake_GoodFeature=(pl.col("Decision_Outcome") == "Accepted") * 0.9
    + pl.Series([random.random() for _ in range(hds_data.height)]) * 0.1,
    DataLake_GoodFeatureCorrelated=(pl.col("Decision_Outcome") == "Accepted") * 0.8
    + pl.Series([random.random() for _ in range(hds_data.height)]) * 0.1
).select(
    [
        pl.col("Customer_ID"),
        pl.col("Decision_DecisionTime").dt.truncate("1d").alias("SnapshotTime"),
        pl.col("DataLake_BadFeature"),
        pl.col("DataLake_GoodFeature"),
        pl.col("DataLake_GoodFeatureCorrelated")
    ]
).group_by(
    ["Customer_ID", "SnapshotTime"]
).agg(
    cs.all().mean()
).sort(["Customer_ID", "SnapshotTime"])

datalake_fake_data.head()

Customer_ID,SnapshotTime,DataLake_BadFeature,DataLake_GoodFeature,DataLake_GoodFeatureCorrelated
str,datetime[ns],f64,f64,f64
"""Customer-1253""",2021-05-24 00:00:00,0.551759,0.178849,0.155624
"""Customer-1254""",2021-05-24 00:00:00,0.592678,0.179437,0.161329
"""Customer-1255""",2021-05-24 00:00:00,0.506912,0.092285,0.090262
"""Customer-1256""",2021-05-24 00:00:00,0.399738,0.057378,0.043184
"""Customer-1257""",2021-05-24 00:00:00,0.494478,0.043562,0.049615


Joining that data with the HDS data is straightforward: we match by customer ID and timestamp, but need to be careful to avoid leakage, so we only join in data for a particular customer from the data lake that is the latest snapshot before the timestamp of the HDS dataset. 

Polars provides a convenient way to do this with the join_asof function.

In [9]:
augmented_data = hds_data.join_asof(
    datalake_fake_data,
    left_on="Decision_DecisionTime",
    right_on="SnapshotTime",
    by="Customer_ID",
)
augmented_data_dictionary = pl.concat(
    [
        hds_data_dictionary,
        pl.DataFrame(
            {
                "Field": ["DataLake_BadFeature", "DataLake_GoodFeature", "DataLake_GoodFeatureCorrelated"],
                "Numeric": [True, True, True],
                "Category": ["DataLake", "DataLake", "DataLake"],
            }
        ),
    ]
)

/var/folders/9w/xtb0b1l95mz5g1vy6xm_48f80000gp/T/ipykernel_56928/3474530575.py:1: UserWarning:

Sortedness of columns cannot be checked when 'by' groups provided



In [10]:
X_train, X_test, y_train, y_test, target_encoder, feature_cols = data_prep(augmented_data, augmented_data_dictionary)
classifier = create_classifier(X_train, X_test, y_train, y_test, target_encoder)
test = show_feature_imp(classifier, X_train, feature_cols)

Categorical fields: ['Context_Name', 'Context_Issue', 'Context_Group', 'Context_Direction', 'Context_Channel', 'Customer_CLV', 'Customer_pyCountry', 'Customer_RiskCode', 'Customer_City', 'Customer_ID', 'Customer_BusinessSegment', 'Customer_Incarceration', 'Customer_LastReviewedDate', 'Customer_ReviewDate', 'Customer_WinScore', 'Customer_InCollections', 'Customer_Age', 'Customer_BalanceTransaction', 'Customer_TotalAssets', 'Customer_IsCustomerActive', 'Customer_HealthMatter', 'Customer_MilitaryService', 'Customer_Bankruptcy', 'Customer_RiskScore', 'Customer_State', 'Customer_NaturalDisaster', 'Customer_MaritalStatus', 'Customer_AnnualIncome', 'Customer_NextReviewDate', 'Customer_TotalLiabilities', 'Customer_ResidentialStatus', 'Customer_OrganizationLabel', 'Customer_pyRegion', 'Customer_Deceased', 'IH_Email_Outbound_Rejected_pxLastGroupID', 'IH_Web_Inbound_Accepted_pxLastGroupID', 'IH_SMS_Outbound_Loyal_pxLastGroupID', 'IH_Email_Outbound_Accepted_pxLastGroupID', 'IH_Web_Inbound_Churned_

## Conclusions

The resulting feature importance shows how the new "GoodFeature" and its correlated variant are on top of the list of features, and the "BadFeature" is at the bottom. You should consider the new features with high feature importance for inclusion in Pega.

We also do a correlation check, so very similar features are labeled with their group index. In this example, you would want only the best feature of "Group 1" to be included, the other one won't add much values since it is so highly correlated.
